# Pytorch Model Generation (Pothole Detection) using Yolov5

## Installation
clone yolov5 from ultralytics github repo

In [1]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


change directory and install all dependencies

In [2]:
%cd yolov5
#!python3 -m pip install -qr requirements.txt wandb  # install

/Users/danishshahzad/Documents/GitHub/Pothole-App/src/yolov5


In [3]:
#!python3 -m pip install matplotlib

## Import packages

In [4]:
import torch
from IPython.display import Image, clear_output
import yaml
from IPython.core.magic import register_line_cell_magic

In [5]:
#clear_output()
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.9.1 CPU


## Data Pepration
Every dataset needs to be converted into yolov5 desired format (coco) before merging. Following is the dataset scheme and respective structure

    data_root_dir
        test
            images
            labels
        
        train
            images
            labels
        
        valid
            images
            labels
    data.yaml
    log.txt




In [6]:
%cat ../../data_root_dir/data.yaml

train: ../../data_root_dir/train/images
val: ../../data_root_dir/valid/images

nc: 1
names: ['pothole']

In [7]:
%cat ../../data_root_dir/log

Train 2133  -- 80%

Valid 226   -- 15%

Test 161    -- 05%


## Create Yolov5s template

In [8]:
with open("../../data_root_dir/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [9]:
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [10]:
%%writetemplate models/pothole_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
!python3 train.py --project pothole_detection --epochs 10 --name yolov5s_pothole_results --data ../../data_root_dir/data.yaml --cfg models/pothole_yolov5s.yaml --cache

wandb: Currently logged in as: danish87 (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=models/pothole_yolov5s.yaml, data=../../data_root_dir/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=pothole_detection, name=yolov5s_pothole_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-25-g15e8c4c torch 1.9.1 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_

Once model is genrated you can run following test command to check the performance

In [ ]:
!python3 detect.py --weights pothole_detection/yolov5s_pothole_results/weights/best.pt --conf 0.4 --source ../../data_root_dir/test/images


Display some images from experiment folder

In [ ]:
import glob
from IPython.display import Image, display
num_image=0
for imageName in glob.glob('runs/detect/exp/*.jpg'): #assuming JPG
    if num_image<10:
      display(Image(filename=imageName))
      print(str(num_image) + "\n")
    num_image=num_image+1